In [27]:
system"c 40 200"

In [54]:
complete:("PJFFFFFFFFFJFJJF";enlist ",")0:`$":complete.csv";

In [55]:
final:select from complete where WEEKDAY<5,9<HOUR,HOUR<20

In [58]:
final:select from final where 40 <= (avg;traffic_load) fby traffic_station

In [59]:
final:update traffic_load:traffic_load%100 from final

In [60]:
/ comp2 no migrado de momento

In [263]:
prepareData:{[tt;data]
    op:$[tt=`train;#;_];                                      / `train or `test decide the operator
    m:`rainfall`temperature`traffic_load`HOUR`WEEKDAY;        / the 5 columns we need
    data:?[data;();`traffic_station;m!({((y;80);x)}[;op]')m]; / functional form. essentially we either pick the first 80 elems or from 80 until the end (# or _)
    sw:{({y#z _x}[x;5;]')til count b:10_x};                   / sliding window function. takes turbomatrix and divides into chunks of 5x5
    gl:{6_(flip x)[2]};                                       / gets the load (y data)
    toMatrix:{({[t;i]value t[i]}[x;]')til count x:flip x};    / table to matrix
    data:(toMatrix')data;                                     / convert each subtable (data is a keyed table) to a matrix
    X:(sw')data;                                              / apply sliding window to get X
    y:(gl')data;                                              / and gl to get y
    (X;y)                                                     / return both
    }

In [264]:
train:prepareData[`train;final]

In [265]:
train[0][3403][0]

0f 7.9  0.4975 10 3
0f 9.7  0.525  11 3
0f 11.5 0.4725 12 3
0f 12.7 0.4975 13 3
0f 13.8 0.58   14 3


In [266]:
train[1][3403]

0.4875 0.4725 0.425 0.475 0.5425 0.4725 0.4525 0.4675 0.58 0.4225 0.3925 0.4925 0.5625 0.5775 0.3925 0.485 0.4325 0.57 0.49 0.3325 0.37 0.4325 0.4425 0.4475 0.26 0.445 0.355 0.44 0.4775 0.33 0.1975..


In [267]:
test:prepareData[`test;final]

In [269]:
test[0][3403][0]

0f 13.5 0.4175 11 1
0f 13.7 0.465  12 1
0f 13.7 0.575  13 1
0f 14.1 0.465  14 1
0f 14.4 0.525  15 1


In [270]:
test[1][3403]

0.495 0.4375 0.5325 0.465 0.47 0.455 0.4375 0.51 0.465 0.3875 0.4825 0.4675 0.475 0.52 0.45 0.4675 0.4725 0.4525 0.4675 0.4525 0.5775 0.54 0.445 0.535 0.475 0.43 0.52 0.5175 0.4225 0.375 0.54 0.46 ..
